In [122]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [123]:
from string import translate, maketrans

In [142]:
home = expanduser("~")
annee="18"
def getSerie(nomKalaba):
    return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/%s-%s/"%(annee,nomKalaba)

prevSerie=getSerie("K5")
nextSerie=getSerie("K4")

nomsPropres=["Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["kaleb","maira","vjolet","nabil","katiSa","nikol"]


In [125]:
with open(prevSerie+"Stems.yaml", 'r') as stream:
    prevStems=yaml.load(stream)
with open(nextSerie+"Stems.yaml", 'r') as stream:
    nextStems=yaml.load(stream)

In [126]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [127]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(nextStems)

In [128]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [129]:
with open(prevSerie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)

In [130]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [143]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical,choice="apophonie"):
    result=list(radical)
    cibles=[]
    if not choice:
        choice=rd.choice(["apophonie","rdVowel"])
    if choice=="apophonie":
        for p,l in enumerate(radical):
            if l in "iae":
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=phonology["apophonies"][result[cible]]
            return "".join(result)
        else:
            return None
    else:
        for p,l in enumerate(radical):
            if l in voyelles:
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=rd.choice(voyelles)
            return "".join(result)
        else:
            return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [184]:
apophonie("kulubu",choice="rnd")

'kulabu'

In [132]:
operations=[devoiserFinale,apophonie,voiserVCV,metathese,changerLongueur]

In [133]:
def modifierRadical(radical):
    nOperations=rd.choice([0,1,2,2,3,3,3,4,5])
    while nOperations>0:
        operation=rd.choice(operations)
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    return radical

In [134]:
racineChanges={}
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(prevLexique[mot])
    else:
        print "***"
        print mot,"n'est pas dans le nouveau lexique"
        print "***"

for mot in nextLexique:
    if not mot in prevLexique:
        print "***"
        print mot,u"n'était pas dans le lexique précédent"
        print "***"

In [135]:
modifierRadical("Tonif")

'Tofan'

In [136]:
racineChanges

{'Dazak': 'zuDuk',
 'Dudud': 'dutuD',
 'Duguru': 'Durugu',
 'Durag': 'Tukur',
 'Nanop': 'Nubon',
 'Norawo': 'rowuNu',
 'Norumu': 'Nomur',
 'NuZad': 'ZiNid',
 'Nuwu': 'Nuwi',
 'Nuzoto': 'Natez',
 'Sas': 'zaS',
 'Sazuf': 'Sizuv',
 'Serun': 'roSunu',
 'Sinez': 'Sinesu',
 'SubuZe': 'SubuSe',
 'Surup': 'Surupu',
 'Sutok': 'Zugodu',
 'TaZuk': 'TuZuk',
 'Tatod': 'Dudot',
 'Tibiwi': 'Tibawu',
 'Toko': 'kuj',
 'Tuduf': 'Tutuf',
 'Tumot': 'muDot',
 'Zefud': 'deZufa',
 'Zeluf': 'leZuvu',
 'ZiNaf': 'ZaNuf',
 'Zogol': 'goZola',
 'Zuluvo': 'Zaluvo',
 'bakol': 'kupel',
 'baleDo': 'piloT',
 'biZaS': 'ZiSab',
 'boNot': 'Nobot',
 'budug': 'budug',
 'daNufi': 'duNafi',
 'daboZ': 'tipoZu',
 'dovuz': 'defaz',
 'dowuf': 'wetuf',
 'dugul': 'gudulu',
 'dujo': 'tujo',
 'dujon': 'nujet',
 'fiTip': 'Tavip',
 'fobak': 'bofuk',
 'fomula': 'lofum',
 'fuNub': 'fubuN',
 'fuNuso': 'vuNus',
 'gawapa': 'wigap',
 'ges': 'sogoZ',
 'gobeva': 'gobovu',
 'guDuN': 'DuguN',
 'gutun': 'gunuto',
 'guwar': 'wugure',
 'jvolo': 'jv

In [137]:
newStems=replace_keys(nextStems,racineChanges)

In [138]:
with open(nextSerie+'flowStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [139]:
with open(nextSerie+"flowStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [140]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  A1:
    Nobot: [trois]
    TuZuk: [bas, basse, basses]
    gudulu: [gros, grosse, grosses]
    guzup: [grand, grande, grands, grandes]
    kikow: [profond, profonds, profonde, profondes]
    nunuz: [rouge, rouges]
    pevip: [blessé, blessés, blessée, blessées]
    rowuNu: [terrible, terribles]
    sorawu: [furieux, furieuse, furieuses]
    zubobe: [noir, noire, noirs, noires]
  A2:
    Dudot: [maigre, maigres]
    TiwuNo: [jaune, jaunes]
    burufa: [effrayé, effrayés, effrayée, effrayées]
    muDot: [suivant, suivants, suivante, suivantes]
    nitigu: [courageux, courageuse, courageuses]
    pavup: [blanc, blanche, blancs, blanches]
    roTuf: [petit, petite, petits, petites]
    rudoz: [différent, différents, différente, différentes]
    rufop: [quatre]
DET:
  b: [DEM, ce, cet, cette, ces]
  kuj: [DEF, le, la, les, l]
  lu: [IND, un, une, des, dE]
NOM:
  N1:
    N11:
      A:
        Tibawu: [forêt, forêts]
        sopap: [coup, coups]
        tujur: [infirmière, infirmières]

In [141]:
with open(nextSerie+'newStems.yaml', 'w') as output:
    output.write(yamlText)